Data Co-relation between crypto, snp500, Inflation - Vish

Data loading, Data Exploring, Data cleaning, Data Wrangling - Eromosele

Data Splitting, Feature Engineering - Scree


Model Training - Max


Model Optimization - Lakshya